In [1]:
# If the libraries are not yet installed, they can be installed using conda commands similar to the below
# %conda install numpy
# %conda install pandas
# %conda install pandasql 
# %conda install openpyxl

# Something like the following may also work if the above does not
# import sys
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} pandasql
# !conda install --yes --prefix {sys.prefix} openpyxl

# To install a specific version, add the version to the install command
# E.g., %conda install numpy=1.20.3

# If all else fails, use pip or follow additional advice such as found at
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

# If your plan to use pip (especially if you are not working within a specified conda environment), 
# the pip commands might look like:
# pip install numpy
# pip install pandas
# pip install pandasql
# pip install open pyxl

# To install a specific version, add the version to the pip install command
# E.g., pip install numpy==1.20.3

In [2]:
# Load necessary libraries
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import pandasql as psql
import glob
import time
import os

In [3]:
# Create intermediate and processed directories for later use. We use the makedirs method because this can 
# create the directories recursively, and we set exist_ok to True to handle the case where the directories
# already exist
os.makedirs("Data/Intermediate/Highways_England", exist_ok=True)
os.makedirs("Data/Intermediate/Utah", exist_ok=True)
os.makedirs("Data/Intermediate/Portland", exist_ok=True)

os.makedirs("Data/Processed/Highways_England", exist_ok=True)
os.makedirs("Data/Processed/Utah", exist_ok=True)
os.makedirs("Data/Processed/Portland", exist_ok=True)

# Functions to Read in Raw Data and Impute with Linear Interpolation

In [4]:
def full_time(df, interval="15min", min_date=None, max_date=None):
    """Function to generate full df of dates to determine which ones have missing data"""
    
    # Specify the start and end timestamps for the full date range
    if not min_date:
        start_date = df["timestamp"].min()
    else:
        start_date = min_date
    
    if not max_date:
        end_date = df["timestamp"].max()
    else:
        end_date = max_date
    
    # Create a full date range using specified freq, typically either 15min or 5 min depending on the data 
    date_list = pd.date_range(start=start_date, end=end_date, freq=interval)
    
    # Put this date range into a df
    date_df = pd.DataFrame({"timestamp": date_list})
    
    # Create columns for date, day of week, and day of year in addition to the timestamp column
    date_df["date"] = pd.to_datetime(date_df["timestamp"].astype("string").str[:10])
    date_df["day_of_week"] = date_df["date"].dt.dayofweek
    date_df["day_of_year"] = date_df["date"].dt.dayofyear
    
    return date_df

In [5]:
def read_portland(fname, highway_number, highway_id, min_date=None, max_date=None):
    """Function to read in data from a full highway in Portland/Vancouver and return a df with only
    the detector with best data availability and only necessary columns"""
    
    # Read in the csv with pandas
    df = pd.read_csv(fname)
    
    # Add date field
    df["date"] = pd.to_datetime(df["starttime"].astype("str").str[:10])
    df["mnth"] = df["date"].dt.to_period("M")

    if highway_id == '50': 
        # Highway 50 has several good detectors, so we specify that we want 102003
        best_det = 102003
    elif highway_id == '10':
        # For each of the elif statements, we are specifying the specific detector 
        # because these highways have more than one "best detector" in terms of data availability
        best_det = 100300
    elif highway_id == '12':
        best_det = 100650
    elif highway_id == '1':
        best_det = 100703
    elif highway_id == '2':
        best_det = 100688
    elif highway_id == '51':
        best_det = 102001
    elif highway_id == '52':
        best_det = 1000022
    elif highway_id == '53':
        best_det = 1000104
    elif highway_id == '618':
        best_det = 101745
    elif highway_id == '6':
        best_det = 100395
    elif highway_id == '7':
        best_det = 101108
    elif highway_id == '8':
        best_det = 101161
    elif highway_id == '9':
        best_det = 100314
    else:  
        # If we have not specified a detector, we group by detector and count, sort, 
        # and grab the last detector in the object (this will have the highest count)
        best_det = df.groupby(["detector_id"]).count().reset_index()[["detector_id", "volume"]]\
        .sort_values(by="volume").iloc[-1,0]

    # Subset the dataframe to only this detector and return the subset
    df_sub = df.query("detector_id == @best_det").copy()

    # Set the site name in the df
    df_sub["site_name"] = f"{highway_number}/" + df_sub["detector_id"].astype("str") 

    # Get the time period ending in the same format as highways England    
    df_sub["time_period_start"] = df_sub["starttime"].astype("str").str[11:19]
    df_sub["time_period_ending"] = (pd.to_datetime(df_sub["time_period_start"]) + datetime.timedelta(minutes=14)).dt.time.astype("str")
    df_sub["timestamp"] = pd.to_datetime(df_sub["date"].astype("str") + " " + df_sub["time_period_ending"])

    # Rename columns
    df_sub["total_volume"] = df_sub["volume"]
    df_sub["avg_mph"] = df_sub["speed"]

    # Grab only needed columns
    df_sub = df_sub[["site_name", "timestamp", "avg_mph", "total_volume"]]

    dates = full_time(df_sub, interval="15min", min_date=min_date, max_date=max_date)

    # Merge full date list with actual data
    df = dates.merge(df_sub, how="left", on="timestamp")

    site = df["site_name"].unique()[0]

    df.fillna({"site_name": site}, inplace=True)

    # Use pandasql to impute the 'interval_of_day' field 
    interval_of_day_impute = """
    SELECT site_name,
           day_of_week,
           date(date) AS date,
           day_of_year,
           timestamp,
           DENSE_RANK() OVER (PARTITION BY DATE ORDER BY timestamp) - 1 AS interval_of_day,
           avg_mph,
           total_volume
    FROM df
    """
    df = psql.sqldf(interval_of_day_impute, locals())

    # Create field with T/F if speed data is missing
    df["missing_speed"] = np.where(df["avg_mph"].isnull(), True, False)

    # Create field with T/F if volume data is missing
    df["missing_volume"] = np.where(df["total_volume"].isnull(), True, False)

    # Use linear interoplation to fill in nulls
    df["avg_mph"] = df["avg_mph"].interpolate()
    df["total_volume"] = df["total_volume"].interpolate()

    df = df.drop_duplicates()

    return df, best_det

In [6]:
def read_highways_england(fname, min_date=None, max_date=None):
    """Function to read in csv file of highway sensor data"""
    
    # Read file into Pandas df
    df = pd.read_csv(fname)
    
    # Grab relevant columns from df
    df = df[["Site Name", "Report Date", "Time Period Ending", "Time Interval", "Avg mph", "Total Volume"]]
    
    # Re-format date field and cast to string
    df["Date"] = pd.to_datetime(df["Report Date"], format='%d/%m/%Y 00:00:00').astype("string") 
    
    # Grab the timestamp of the time-period in the hour
    df["Time Period Ending"] = df["Time Period Ending"].astype("string")
    
    # Create a true timestamp which includes both date and hour and minutes
    df["Timestamp"] = pd.to_datetime(df["Date"] + " " + df["Time Period Ending"])
    
    # Subset columns and rename to include _ to make columns easier to work with
    df = df[["Site Name", "Timestamp", "Time Interval", "Avg mph", "Total Volume"]]\
    .rename(columns={"Site Name": "site_name",
                     "Timestamp":"timestamp",
                     "Time Interval": "interval_of_day",
                     "Avg mph": "avg_mph",
                     "Total Volume": "total_volume"})
    
    # Compute dates for left join 
    dates = full_time(df, interval="15min", min_date=min_date, max_date=max_date)
    
    # Merge full date list with actual data
    df = dates.merge(df, how="left", on="timestamp")
    
    site = df["site_name"].unique()[0]
    
    df.fillna({"site_name": site}, inplace=True)
    
    # Use pandasql to impute the 'interval_of_day' field 
    interval_of_day_impute = """
    SELECT site_name,
           day_of_week,
           date(date) AS date,
           day_of_year,
           timestamp,
           DENSE_RANK() OVER (PARTITION BY DATE ORDER BY timestamp) - 1 AS interval_of_day,
           avg_mph,
           total_volume
    FROM df
    """
    df = psql.sqldf(interval_of_day_impute, locals())
    
    # Create field with T/F if speed data is missing
    df["missing_speed"] = np.where(df["avg_mph"].isnull(), True, False)
    
    # Create field with T/F if volume data is missing
    df["missing_volume"] = np.where(df["total_volume"].isnull(), True, False)

    # Set DateTime Index
#     df["timestamp"] = pd.to_datetime(df["timestamp"])
#     df = df.set_index("timestamp")
    
    # Use linear interoplation to fill in nulls
    df["avg_mph"] = df["avg_mph"].interpolate()
    df["total_volume"] = df["total_volume"].interpolate()
    
#     df = df.reset_index()
    
    return df

In [7]:
def read_utah(folder_path, highway_number, detector_id, min_date=None, max_date=None):
    """Function to read in data from a UTAH PEMS sensor and aggregate from 5min to 15min to be the same as 
    other data sources in our set (Highways England, Portland-Vancouver Regional)"""
    
    # Grab all excel files in the provided path, there are typically 13-14 files per sensor and year
    flist = glob.glob(f"{folder_path}/*.xlsx")
    df = pd.DataFrame()
    for fname in flist:
        df_sub = pd.read_excel(fname, engine="openpyxl")
        df = df.append(df_sub)
    df = df.drop_duplicates()
    
    df["date"] = df["5 Minutes"].astype("str").str[0:11]
    df["time_period_start"] = df["5 Minutes"].astype("str").str[11:19]
    df["time_period_ending"] = (pd.to_datetime(df["time_period_start"]) + datetime.timedelta(minutes=4)).dt.time.astype("str")
    df["timestamp"] = pd.to_datetime(df["date"] + " " + df["time_period_ending"]).round("min")
    df["total_volume"] = df["Flow (Veh/5 Minutes)"]
    df["avg_mph"] = df["Speed (mph)"]
    df["site_name"] = f"{highway_number}/{detector_id}"
    
    df = df[["site_name", "timestamp", "avg_mph", "total_volume", "time_period_start"]].copy()
    
    df = df.query("timestamp >= '2019-01-01 00:00:00'").query("timestamp < '2020-01-01 00:00:00'").copy()
    
    date_df = full_time(df, interval="5min", min_date=min_date, max_date=max_date)
    
    full_df = date_df.merge(df, how="left", on="timestamp")
    
    site_name = full_df.site_name.unique()[0]
    full_df.fillna({"site_name": site_name}, inplace=True)
    
    interval_of_day = """
    SELECT  site_name,
            day_of_week,
            date(date) AS date,
            day_of_year,
            timestamp,
            DENSE_RANK() OVER (PARTITION BY DATE ORDER BY timestamp) - 1 AS interval_of_day,
            avg_mph,
            total_volume 
    FROM full_df
    """
    full_df = psql.sqldf(interval_of_day, locals())
    
    # Create field with T/F if speed data is missing
    full_df["missing_speed"] = np.where(full_df["avg_mph"].isnull(), True, False)
    
    # Create field with T/F if volume data is missing
    full_df["missing_volume"] = np.where(full_df["total_volume"].isnull(), True, False)
    
    # Use linear interoplation to fill in nulls
    df["avg_mph"] = df["avg_mph"].interpolate()
    df["total_volume"] = df["total_volume"].interpolate()
        
    # Aggregate up to 15 minute intervals - this section is moved to R because of linear interp not working well
#     full_df["min_15_int"] = full_df["interval_of_day"] // 3
    
#     full_df = full_df.groupby(["site_name", "day_of_week", "date", "day_of_year","min_15_int"]).\
#     agg({"timestamp": "max", "total_volume": "sum", "avg_mph": "mean", 
#          "missing_speed": "max", "missing_volume": "max"}).reset_index()
    
#     full_df.rename(columns={"min_15_int": "interval_of_day"}, inplace=True)
    
    full_df = full_df[["site_name",
                       "day_of_week",
                       "date",
                       "day_of_year",
                       "timestamp",
                       "interval_of_day",
                       "avg_mph",
                       "total_volume",
                       "missing_speed",
                       "missing_volume"]].copy()
    
    return full_df

# Read in Raw Data, Impute, and Write to New Directory

In [8]:
# Set the min and max dates for the 5-min and 15-min data
min_date_5 = "2019-01-01 00:04:00"
min_date_15 = "2019-01-01 00:14:00"
max_date = "2019-12-31 23:59:00"

In [9]:
# Initialize a counter
c = 0

In [10]:
# Read in all raw highways england data files, impute, and write to the intermediate direcotry of the data folder
for fname in glob.glob("Data/Raw/Highways_England/*.csv"):
    print("Reading {}".format(fname))
    
    fname_new = "Data/Intermediate/Highways_England/{}_Intermediate.csv".format(fname.split("/")[-1].split(".")[0])
    
    df = read_highways_england(fname, min_date_15, max_date)
    
    df.to_csv(fname_new, index=False)
    
    c += 1

Reading Data/Raw/Highways_England/A11-6310-1_Southbound_2019.csv
Reading Data/Raw/Highways_England/A11-6312-2_Northbound_2019.csv
Reading Data/Raw/Highways_England/A14-1107A_Eastbound_2019.csv
Reading Data/Raw/Highways_England/A14-1144B_Westbound_2019.csv
Reading Data/Raw/Highways_England/A1M-9842B_Southbound_2019.csv
Reading Data/Raw/Highways_England/A1M-9847a_Northbound_2019.csv
Reading Data/Raw/Highways_England/A46-7636-1_Northbound_2019.csv
Reading Data/Raw/Highways_England/A46-7636-2_Southbound_2019.csv
Reading Data/Raw/Highways_England/A47-6337-1_Westbound_2019.csv
Reading Data/Raw/Highways_England/A47-6337-2_Eastbound_2019.csv
Reading Data/Raw/Highways_England/A5-6847-2_Southbound_2019.csv
Reading Data/Raw/Highways_England/A5-7572-1-Northbound_2019.csv
Reading Data/Raw/Highways_England/A590-9531-1_Eastbound_2019.csv
Reading Data/Raw/Highways_England/A590-9634-1_Westbound_2019.csv
Reading Data/Raw/Highways_England/A64-9251-1_Westbound_2019.csv
Reading Data/Raw/Highways_England/A6

In [11]:
# Read in the highway id metadata for the portland regional highway system
portland_highway_numbers = pd.read_csv("Data/Metadata/portland_highway_metadata.csv")
portland_highway_numbers['highwayid'] = portland_highway_numbers['highwayid'].astype("str")

# Read in all raw portland data files, impute, and write to the intermediate direcotry of the data folder
for fname in glob.glob("Data/Raw/Portland/*.csv"):
    print("Reading {}".format(fname))
    
    # Parse the highway name and id in order to properly name the new file
    highway_id = fname.split("highway")[-1].split("_")[0]
    highway_name = portland_highway_numbers.query("highwayid==@highway_id")["highwayname"].iloc[0].replace("-", "")
    direction = portland_highway_numbers.query("highwayid==@highway_id")["direction"].iloc[0].capitalize()
    
    # Read in each raw file
    df, det = read_portland(fname, highway_name, highway_id, min_date=min_date_15, max_date=max_date)
    
    fname_new = "Data/Intermediate/Portland/{}-{}_{}bound_2019_Intermediate.csv".format(highway_name, det, direction)
    
    # Write the processed file
    df.to_csv(fname_new, index=False)
    
    c += 1

Reading Data/Raw/Portland/all_sensors_highway10_portland.csv
Reading Data/Raw/Portland/all_sensors_highway11_portland.csv
Reading Data/Raw/Portland/all_sensors_highway12_portland.csv
Reading Data/Raw/Portland/all_sensors_highway1_portland.csv
Reading Data/Raw/Portland/all_sensors_highway2_portland.csv
Reading Data/Raw/Portland/all_sensors_highway3_portland.csv
Reading Data/Raw/Portland/all_sensors_highway4_portland.csv
Reading Data/Raw/Portland/all_sensors_highway50_portland.csv
Reading Data/Raw/Portland/all_sensors_highway51_portland.csv
Reading Data/Raw/Portland/all_sensors_highway52_portland.csv
Reading Data/Raw/Portland/all_sensors_highway53_portland.csv
Reading Data/Raw/Portland/all_sensors_highway5_portland.csv
Reading Data/Raw/Portland/all_sensors_highway618_portland.csv
Reading Data/Raw/Portland/all_sensors_highway620_portland.csv
Reading Data/Raw/Portland/all_sensors_highway621_portland.csv
Reading Data/Raw/Portland/all_sensors_highway6_portland.csv
Reading Data/Raw/Portland/a

In [12]:
# Read in all raw utah data files, impute, and write to the intermediate directory of the data folder
for path in glob.glob("Data/Raw/Utah/*"):
    print("Reading files in {} directory".format(path))
    
    highway_number = path.split("/")[-1].split("_")[0]
    detector_id = path.split("/")[-1].split("_")[1]
    direction = path.split("/")[-1].split("_")[2]
    
    df = read_utah(path, highway_number, detector_id, min_date=min_date_5, max_date=max_date)
    
    fname_new = "Data/Intermediate/Utah/{}-{}_{}_2019_Intermediate.csv".format(highway_number, detector_id, direction)
    
    df.to_csv(fname_new, index=False)
    
    c += 1

Reading files in Data/Raw/Utah/I15_3103178_Southbound directory
Reading files in Data/Raw/Utah/I15_749_Northbound directory
Reading files in Data/Raw/Utah/I215_134_Counterclockwise directory
Reading files in Data/Raw/Utah/I215_31_Clockwise directory
Reading files in Data/Raw/Utah/I70_3103400_Westbound directory
Reading files in Data/Raw/Utah/I70_3103401_Eastbound directory
Reading files in Data/Raw/Utah/I80_600_Eastbound directory
Reading files in Data/Raw/Utah/I80_667_Westbound directory
Reading files in Data/Raw/Utah/I84_451_Eastbound directory
Reading files in Data/Raw/Utah/I84_482_Westbound directory
Reading files in Data/Raw/Utah/LegacyParkway_810_Northbound directory
Reading files in Data/Raw/Utah/LegacyParkway_890_Southbound directory
Reading files in Data/Raw/Utah/US189_260_Westbound directory
Reading files in Data/Raw/Utah/US189_470_Eastbound directory
Reading files in Data/Raw/Utah/US40_634_Westbound directory
Reading files in Data/Raw/Utah/US40_635_Eastbound directory
Readin

In [13]:
# Sanity check the counter
c

76